In [33]:
import pandas as pd
import glob

# # Define the label mapping
# activity_labels = {
#     'walking': 1,
#     'walking_upstair': 2,
#     'walking_downstair': 3,
#     'sitting': 4,
#     'standing': 5,
#     'laying': 6
# }

# # List all CSV files in the directory
# file_paths = glob.glob('data+scripts/our_data/*.csv')

# # Initialize empty lists to store the data and labels
# X_test_list = []
# y_test_list = []

# # Iterate through each file
# for file_path in file_paths:
#     # Extract the activity name from the file name
#     file_name = file_path.split('/')[-1].lower()
#     activity_name = file_name.split('_')[-1].replace('.csv', '')

#     # Determine the label for the activity
#     label = activity_labels.get(activity_name)
    
#     if label is not None:
#         # Read the CSV file
#         df = pd.read_csv(file_path)
        
#         # Take only alternate rows and limit to 500 rows
#         df_alternate = df.iloc[::2, :]
        
#         # Add the data and corresponding label to the lists
#         X_test_list.append(df_alternate)
#         y_test_list.extend([label] * len(df_alternate))

# # Concatenate all dataframes to form X_test
# X_test = pd.concat(X_test_list, ignore_index=True)

# # Convert the labels list to a DataFrame to form y_test
# y_test = pd.DataFrame(y_test_list, columns=['Activity'])

# # Example: Displaying the first few rows of X_test and y_test
# print("First few rows of X_test:")
# print(X_test.head())

# print("\nFirst few rows of y_test:")
# print(y_test.head())


In [28]:
# X_test.shape

(15911, 5)

In [ ]:
# Define the label mapping for your custom dataset
activity_labels = {
    'walking': 1,
    'walking_upstair': 2,
    'walking_downstair': 3,
    'sitting': 4,
    'standing': 5,
    'laying': 6
}

# List all CSV files in the directory
file_paths = glob.glob('data+scripts/our_data/*.csv')

# Initialize empty lists to store the data and labels
X_test_list = []
y_test_list = []

# Iterate through each file to prepare test data
for file_path in file_paths:
    # Extract the activity name from the file name
    file_name = file_path.split('/')[-1].lower()
    activity_name = file_name.split('_')[-1].replace('.csv', '')

    # Determine the label for the activity
    label = activity_labels.get(activity_name)
    
    if label is not None:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Apply windowing to match the training data structure
        df_windows = create_windows(df)
        
        # Add the data and corresponding label to the lists
        X_test_list.append(df_windows)
        y_test_list.extend([label] * len(df_windows))

# Concatenate all dataframes to form X_test
X_test = pd.concat(X_test_list, ignore_index=True)

# Convert the labels list to a DataFrame to form y_test
y_test = pd.DataFrame(y_test_list, columns=['Activity'])


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import tsfel
from joblib import Parallel, delayed
from sklearn.metrics import accuracy_score

In [9]:
# Define file paths for training data (raw accelerometer data)
axes = ["x", "y", "z"]
file_paths = {axis: f"data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/train/Inertial Signals/total_acc_{axis}_train.txt" for axis in axes}

# Load data for each axis into a dictionary of DataFrames
dfs = {axis: pd.read_csv(file_paths[axis], header=None, delim_whitespace=True) for axis in axes}

# Compute the magnitude of acceleration: sqrt(x^2 + y^2 + z^2)
df_magnitude = pd.DataFrame(np.sqrt(dfs['x']**2 + dfs['y']**2 + dfs['z']**2))

# Function to create windows of a specific size (128)
def create_windows(df, window_size=128):
    windows = []
    for i in range(0, len(df) - window_size + 1, window_size):
        window = df.iloc[i:i+window_size, :]
        windows.append(window.values.flatten())  # Flatten the window to match model input
    return pd.DataFrame(windows)

# Apply windowing to the training data
df_train_acc = create_windows(df_magnitude)

# Load the labels for training data
y_train = pd.read_csv("data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/train/y_train.txt", header=None)

C:\Users\SIDDHARTH\AppData\Local\Temp\ipykernel_14952\534280910.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfs = {axis: pd.read_csv(file_paths[axis], header=None, delim_whitespace=True) for axis in axes}
C:\Users\SIDDHARTH\AppData\Local\Temp\ipykernel_14952\534280910.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfs = {axis: pd.read_csv(file_paths[axis], header=None, delim_whitespace=True) for axis in axes}
C:\Users\SIDDHARTH\AppData\Local\Temp\ipykernel_14952\534280910.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfs = {axis: pd.read_csv(file_paths[axis], header=None, delim_whitespace=True) for axis in axes}


In [29]:
# # Load the combined acceleration magnitude data for testing (repeat the magnitude calculation process for test data)
# axes = ["x", "y", "z"]
# file_paths_test = {axis: f"data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals/total_acc_{axis}_test.txt" for axis in axes}

# # Load data for each axis into a dictionary of DataFrames for the test set
# dfs_test = {axis: pd.read_csv(file_paths_test[axis], header=None, delim_whitespace=True) for axis in axes}

# # Compute the magnitude of acceleration for the test set
# df_test_acc = pd.DataFrame(np.sqrt(dfs_test['x']**2 + dfs_test['y']**2 + dfs_test['z']**2))

# # Load the labels for the test data
# y_test = pd.read_csv("data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)


In [31]:
# Assuming df_train_acc and y_train are already loaded
# Train a Decision Tree model
clf = DecisionTreeClassifier()
clf.fit(df_train_acc, y_train.values.ravel())

# Load the test data
# Assuming X_test and y_test are already loaded as per the previous code

# Predict the activities for the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Confusion Matrix:")
print(conf_matrix)

c:\Users\SIDDHARTH\OneDrive\VS\ES335_assignment_1\ML-Assignment-Gamble.ai\.conda\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 5 features, but DecisionTreeClassifier is expecting 128 features as input.

In [30]:
# Train a Decision Tree model
model = DecisionTreeClassifier()
model.fit(df_train_acc, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_test)

c:\Users\SIDDHARTH\OneDrive\VS\ES335_assignment_1\ML-Assignment-Gamble.ai\.conda\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 5 features, but DecisionTreeClassifier is expecting 128 features as input.

In [ ]:
# Calculate accuracy, precision, recall, and confusion matrix
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


In [ ]:
# Load the combined acceleration magnitude data for testing (repeat the magnitude calculation process for test data)
axes = ["x", "y", "z"]
file_paths_test = {axis: f"data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals/total_acc_{axis}_test.txt" for axis in axes}

# Load data for each axis into a dictionary of DataFrames for the test set
dfs_test = {axis: pd.read_csv(file_paths_test[axis], header=None, delim_whitespace=True) for axis in axes}

# Compute the magnitude of acceleration for the test set
df_test_acc = pd.DataFrame(np.sqrt(dfs_test['x']**2 + dfs_test['y']**2 + dfs_test['z']**2))

# Load the labels for the test data
y_test = pd.read_csv("data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)


C:\Users\SIDDHARTH\AppData\Local\Temp\ipykernel_12524\3391091943.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfs_test = {axis: pd.read_csv(file_paths_test[axis], header=None, delim_whitespace=True) for axis in axes}
C:\Users\SIDDHARTH\AppData\Local\Temp\ipykernel_12524\3391091943.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfs_test = {axis: pd.read_csv(file_paths_test[axis], header=None, delim_whitespace=True) for axis in axes}
C:\Users\SIDDHARTH\AppData\Local\Temp\ipykernel_12524\3391091943.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfs_test = {axis: pd.read_csv(file_paths_test[axis], header=None, delim_whitespace=True) for axis in axes}


In [ ]:
# Train a Decision Tree model
model = DecisionTreeClassifier()
model.fit(df_train_acc, y_train)

# Predict the labels for the test set
y_pred = model.predict(df_test_acc)